# Create and Store Embeddings Using BigQuery + FeatureStore

## Import Libraries

In [ ]:
from google.cloud import bigquery, aiplatform
from google.cloud.aiplatform_v1.types import NearestNeighborQuery
from vertexai.resources.preview import feature_store
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

## Set Variables

In [ ]:
project_id = "jesusarguelles-sandbox" # @param {type: "string"}
embeddings_datable = "jesusarguelles-sandbox.demos.hackernews_table_emb" # @param {type: "string"}
feature_store_online_id = "fs_hckn"  # @param {type: "string"}

## Initialization

In [ ]:
aiplatform.init(project=project_id, location="us-central1")
bq_client = bigquery.Client(project=project_id)
emb_model = TextEmbeddingModel.from_pretrained("text-embedding-004")

## Create Table with Embeddings

In [ ]:
# Query recommendted to avoid throttling
sql_query = """
-- *** Updating the fields below is required ***
-- The name of the source table
DECLARE source_table DEFAULT 'bigquery-public-data.bbc_news.fulltext';
-- The name of the target table
DECLARE target_table DEFAULT 'target_dataset.news_body_embeddings';
-- The name of the ML model to use for the ML operation
DECLARE ml_model DEFAULT 'target_dataset.embedding_model';
-- The STRING column from the source table passed to GENERATE_EMBEDDING()
DECLARE content_column DEFAULT 'body';
-- The unique key columns from the source table. These columns are used to identify new rows present
-- in the source table and not the target table. '*' is not supported.
DECLARE key_columns DEFAULT ARRAY['filename'];
-- *** End of section ***

-- *** Updating the fields below is optional ***
-- The number of rows to process per child job. A larger value will reduce the overhead of multiple
-- child jobs, but needs to be small enough to complete in a single job run.
DECLARE batch_size DEFAULT 80000;
-- The time to wait before the script terminates
DECLARE termination_time_secs DEFAULT(23 * 60 * 60);
-- An optional where clause to apply to the source table
DECLARE where_clause DEFAULT 'TRUE';
-- The columns to project from the source table to the target table
DECLARE projection_columns DEFAULT ARRAY['*'];
-- The ML options to use for the ML operation
DECLARE ml_options DEFAULT 'STRUCT(TRUE AS flatten_json_output)';
-- *** End of section ***

-- *** Updating the fields below should be quite rare ***
-- The ML query to use for the ML operation, requires the unique key
DECLARE
  ml_query
    DEFAULT
      FORMAT(
        'SELECT %s, %s AS content FROM `%s` WHERE %s',
        ARRAY_TO_STRING(projection_columns, ','),
        content_column,
        source_table,
        where_clause);

-- The filter condition for accepting the ML result into the target table
DECLARE
  accept_filter
    DEFAULT 'ml_generate_embedding_status' || " NOT LIKE 'A retryable error occurred:%'";

DECLARE
  key_cols_filter
    DEFAULT(
      SELECT
        STRING_AGG('S.' || KEY || ' = T.' || KEY, ' AND ')
      FROM
        UNNEST(key_columns) AS KEY
    );
-- *** End of section ***

-- Create the target table first if it does not exist
EXECUTE
  IMMEDIATE
    FORMAT(
      '''
CREATE TABLE IF NOT EXISTS `%s` AS
  (SELECT *
   FROM ML.GENERATE_EMBEDDING (MODEL `%s`,
           (SELECT *
            FROM (%s)
            LIMIT 10), %s)
   WHERE %s)''',
      target_table,
      ml_model,
      ml_query,
      ml_options,
      accept_filter);

-- Iteratively populate the target table
REPEAT
DROP TABLE IF EXISTS _SESSION.embedding_batch;

-- Identify new rows in the source table to generate embeddings
-- For throughput reasons, materialize these rows into a temp table before calling GENERATE_EMBEDDING()
EXECUTE
  IMMEDIATE
    FORMAT(
      '''
      CREATE TEMP TABLE _SESSION.embedding_batch AS
      (SELECT *
          FROM (%s) AS S
          WHERE NOT EXISTS (SELECT * FROM %s AS T WHERE %s) LIMIT %d)
    ''',
      ml_query,
      target_table,
      key_cols_filter,
      batch_size);

-- Generate embeddings for these rows and insert them into the target table
EXECUTE
  IMMEDIATE
    FORMAT(
      '''
        INSERT `%s`
        SELECT *
            FROM ML.GENERATE_EMBEDDING (MODEL `%s`,
                    TABLE _SESSION.embedding_batch, %s)
            WHERE %s
        ''',
      target_table,
      ml_model,
      ml_options,
      accept_filter);

UNTIL(
  SELECT
    @@row_count
)
= 0
OR TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), @@script.creation_time, SECOND)
  >= termination_time_secs
    END
      REPEAT;
"""

In [ ]:
sql_query = """
CREATE OR REPLACE TABLE `jesusarguelles-sandbox.demos.hackernews_table_emb` AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL `jesusarguelles-sandbox.demos.embeddings`,
  (SELECT text as content FROM jesusarguelles-sandbox.demos.hackernews_table),
  STRUCT(TRUE AS flatten_json_output,
    'RETRIEVAL_DOCUMENT' AS task_type)
);

"""
job = bq_client.query(sql_query)
job.result()

## Creating Features Store from BigQuery

In [ ]:
sql_query = f"""
SELECT * FROM {embeddings_datable}
"""
job = bq_client.query(sql_query)
res = job.result()

In [ ]:
sql_query = f"""
CREATE OR REPLACE TABLE `jesusarguelles-sandbox.demos.hackernews_table_emb_with_id` AS
SELECT
    ml_generate_embedding_result as embedding,
    content,
    CAST(ROW_NUMBER() OVER() AS STRING) AS unique_id
FROM
    `{embeddings_datable}`
WHERE ARRAY_LENGTH(ml_generate_embedding_result) = 768
;
"""
job = bq_client.query(sql_query)
res = job.result()

### Creating Online Store

In [ ]:
fos = feature_store.FeatureOnlineStore.create_optimized_store(
    feature_store_online_id
)

INFO:vertexai.resources.preview.feature_store.feature_online_store:Creating FeatureOnlineStore
INFO:vertexai.resources.preview.feature_store.feature_online_store:Create FeatureOnlineStore backing LRO: projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn/operations/6929764564833665024
INFO:vertexai.resources.preview.feature_store.feature_online_store:FeatureOnlineStore created. Resource name: projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn
INFO:vertexai.resources.preview.feature_store.feature_online_store:To use this FeatureOnlineStore in another session:
INFO:vertexai.resources.preview.feature_store.feature_online_store:feature_online_store = aiplatform.FeatureOnlineStore('projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn')


In [ ]:
sql_query = """
SELECT ARRAY_LENGTH(embedding) AS embedding_dim FROM `jesusarguelles-sandbox.demos.hackernews_table_emb_with_id` LIMIT 1
"""
job = bq_client.query(sql_query)
res = job.result()

In [ ]:
for i in res:
  print(i)

Row((768,), {'embedding_dim': 0})


## Create Feature View Instance

In [ ]:
bigquery_source = feature_store.utils.FeatureViewBigQuerySource(
        uri="bq://jesusarguelles-sandbox.demos.hackernews_table_emb_with_id",
        entity_id_columns=["unique_id"],
    )
index_config = feature_store.utils.IndexConfig(
    embedding_column="embedding",
    dimensions=768,
    algorithm_config=feature_store.utils.TreeAhConfig(),
)

fv = fos.create_feature_view(
    name="hacker_news_f_view",
    source=bigquery_source,
    index_config=index_config,
)

INFO:vertexai.resources.preview.feature_store.feature_online_store:Creating FeatureView
INFO:vertexai.resources.preview.feature_store.feature_online_store:Create FeatureView backing LRO: projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn/featureViews/hacker_news_f_view/operations/1921058091755896832
INFO:vertexai.resources.preview.feature_store.feature_online_store:FeatureView created. Resource name: projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn/featureViews/hacker_news_f_view
INFO:vertexai.resources.preview.feature_store.feature_online_store:To use this FeatureView in another session:
INFO:vertexai.resources.preview.feature_store.feature_online_store:feature_view = aiplatform.FeatureView('projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn/featureViews/hacker_news_f_view')


In [ ]:
sync_response = fv.sync()

In [ ]:
import time

while True:
    feature_view_sync = fv.get_sync(
        sync_response.resource_name.split("/")[9]
    ).gca_resource
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}. \n {feature_view_sync}")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing

In [ ]:
query = "Excellent piece"
inputs = [TextEmbeddingInput(query, "RETRIEVAL_DOCUMENT")]
embeddings = emb_model.get_embeddings(inputs)

In [ ]:
emb=embeddings[0].values

In [ ]:
r = fv.search(
    embedding_value = emb,
    neighbor_count = 5,
    return_full_entity=True,  # returning entities with metadata
).to_dict()

In [ ]:
for n in r["neighbors"]:
  for feature in n["entity_key_values"]["key_values"]["features"]:
    if feature["name"] == "content":
      print(feature["value"])

{'string_value': 'An excellent piece.'}
{'string_value': 'Excellent piece of work.'}
{'string_value': 'Amazing piece of work'}
{'string_value': 'Very well done.'}
{'string_value': 'This is a really good piece. Thanks!'}


In [ ]:
r["neighbors"][0]["entity_id"]

'320373'

In [ ]:
fview = "projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn/featureViews/hacker_news_f_view"
fv = feature_store.FeatureView(name=fview)

In [ ]:
r = fv.search(
    embedding_value = emb,
    neighbor_count = 5,
    return_full_entity=True,  # returning entities with metadata
).to_dict()

INFO:vertexai.resources.preview.feature_store.feature_view:Public endpoint for the optimized online store fs_hckn is 2091622959750840320.us-central1-390227712642.featurestore.vertexai.goog


## List FeatureView

In [ ]:
feature_store.FeatureView.list(feature_online_store_id="fs_hckn")

 resource name: projects/390227712642/locations/us-central1/featureOnlineStores/fs_hckn/featureViews/hacker_news_f_view]